### Step 1: Setting Up the Environment
#### Install the required libraries:

In [2]:
# !pip install tensorflow keras opencv-python scikit-learn flask flask-cors numpy pandas

### Step 2: Download and Prepare the Datasets
#### Mood: Data set : https://www.kaggle.com/datasets/msambare/fer2013
#### Labelled Faces in the Wild (LFW) Dataset : https://www.kaggle.com/datasets/jessicali9530/lfw-dataset?resource=download
#### Age DB: https://www.kaggle.com/datasets/nitingandhi/agedb-database

### Step 3: Training the Models
#### 3.1 Training a Face Detection Model

In [19]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_lfw_data(data_dir):
    X = []
    y = []
    label_map = {}
    current_label = 0

    # Check if the directory exists
    if not os.path.exists(data_dir):
        print(f"Directory {data_dir} does not exist")
        return np.array(X), np.array(y), label_map
    
    for person_name in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person_name)
        if os.path.isdir(person_dir):
            for image_name in os.listdir(person_dir):
                image_path = os.path.join(person_dir, image_name)
                image = cv2.imread(image_path)
                if image is not None:
                    image = cv2.resize(image, (64, 64))  # Resize to 64x64
                    X.append(image)
                    if person_name not in label_map:
                        label_map[person_name] = current_label
                        current_label += 1
                    y.append(label_map[person_name])
                else:
                    print(f"Failed to read image: {image_path}")
    
    if not X:
        print("No images loaded.")
    else:
        print(f"Loaded {len(X)} images from {data_dir}")

    X = np.array(X)
    y = np.array(y)
    
    return X, y, label_map

# Load the LFW dataset
lfw_data_dir = 'Labelled Faces in the Wild (LFW) Dataset/lfw/lfw-deepfunneled'  # Replace with the path to your LFW data directory
X, y, lfw_label_map = load_lfw_data(lfw_data_dir)

if X.size == 0 or y.size == 0:
    print("No data loaded, please check the dataset directory and files.")
else:
    # Normalize the data
    X = X / 255.0

    # Define a simple CNN model for face detection
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(np.unique(y)), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X, y, epochs=22, validation_split=0.2)

    # Evaluate the model on the validation set - not working
    # val_loss, val_accuracy = model.evaluate(val_generator)
    # print("Validation Loss:", val_loss)
    # print("Validation Accuracy:", val_accuracy)
    
    # Save the model
    model.save('face_detection_model.keras') #.h5



Loaded 13233 images from Labelled Faces in the Wild (LFW) Dataset/lfw/lfw-deepfunneled
Epoch 1/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 44s 59ms/step - accuracy: 0.0490 - loss: 8.1422 - val_accuracy: 0.0000e+00 - val_loss: 10.0848
Epoch 2/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - accuracy: 0.0509 - loss: 7.3915 - val_accuracy: 0.0000e+00 - val_loss: 11.6124
Epoch 3/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - accuracy: 0.0750 - loss: 6.9540 - val_accuracy: 0.0000e+00 - val_loss: 12.5301
Epoch 4/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - accuracy: 0.1070 - loss: 6.4077 - val_accuracy: 0.0000e+00 - val_loss: 14.4490
Epoch 5/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 17s 52ms/step - accuracy: 0.1350 - loss: 5.7958 - val_accuracy: 0.0000e+00 - val_loss: 16.5771
Epoch 6/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 17s 52ms/step - accuracy: 0.1940 - loss: 4.8553 - val_accuracy: 0.0000e+00 - val_loss: 19.8555
Epoch 7/25
331/331 ━━━━━━━━━━━━━━━━━━━━ 18s 53ms/step - accuracy: 0.2521 - loss: 3.8399 - val_accuracy:

#### 3.2 Training an Age Prediction Model

In [25]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load AgeDB data
def load_age_data(data_dir):
    X = []
    y = []

    for image_name in os.listdir(data_dir):
        image_path = os.path.join(data_dir, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, (64, 64))  # Resize to 64x64
            X.append(image)
            age = int(image_name.split("_")[0])  # Extract age from the filename
            y.append(age)

    X = np.array(X)
    y = np.array(y)

    return X, y

# Load the AgeDB dataset
agedb_data_dir = 'Age dataset\AgeDB'  # Replace with the path to your AgeDB data directory
X, y = load_age_data(agedb_data_dir)

if X.size == 0 or y.size == 0:
    print("No data loaded, please check the dataset directory and files.")
else:
    # Normalize the data
    X = X / 255.0

    # Define a CNN model for age prediction
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1)  # Single output for age
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # Train the model
    # model.fit(X, y, epochs=10, validation_split=0.2)
    
    # Train the model with early stopping based on validation loss
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model.fit(X, y, epochs=30, validation_split=0.2, callbacks=[early_stopping])

    # Save the model
    model.save('age_prediction_model.keras')


Epoch 1/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 30s 59ms/step - loss: 42875248.0000 - mae: 5458.2280 - val_loss: 9512725.0000 - val_mae: 2072.9163
Epoch 2/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - loss: 25834510.0000 - mae: 4356.7695 - val_loss: 8798625.0000 - val_mae: 2156.3459
Epoch 3/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - loss: 24133652.0000 - mae: 4152.4844 - val_loss: 10019715.0000 - val_mae: 2622.2415
Epoch 4/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 20s 46ms/step - loss: 23815758.0000 - mae: 4114.8193 - val_loss: 8382102.0000 - val_mae: 2144.6680
Epoch 5/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - loss: 24035932.0000 - mae: 4136.8198 - val_loss: 9014564.0000 - val_mae: 2220.6028
Epoch 6/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - loss: 23134036.0000 - mae: 4047.0549 - val_loss: 8292337.0000 - val_mae: 2032.4668
Epoch 7/30
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - loss: 22639680.0000 - mae: 4026.2153 - val_loss: 8446751.0000 - val_mae: 2078.0125
Epoch 8/30
409/409 ━━━━━━━

#### The values "Train Loss" and "Train MAE" represent the performance metrics of your trained age prediction model on the training dataset. Here's what each of them means:

##### Train Loss (Mean Squared Error):

##### The "Train Loss" value of 19117422.0 represents the mean squared error (MSE) of your model's predictions on the training dataset.
##### MSE is calculated by averaging the squared differences between the actual ages and the ages predicted by your model.
##### In this case, a Train Loss of 19117422.0 indicates that, on average, the squared difference between the predicted ages and the actual ages is approximately 19117422.0.
##### Lower values of MSE indicate better model performance, as they suggest that the model's predictions are closer to the actual ages.

#### Train MAE (Mean Absolute Error):

##### The "Train MAE" value of 3591.60791015625 represents the mean absolute error (MAE) of your model's predictions on the training dataset.
##### MAE is calculated by averaging the absolute differences between the actual ages and the ages predicted by your model.
##### In this case, a Train MAE of 3591.60791015625 indicates that, on average, the absolute difference between the predicted ages and the actual ages is approximately 3591.60791015625 years.
##### Lower values of MAE also indicate better model performance, as they suggest that the model's predictions are closer to the actual ages, on average.

#### Interpretation:
##### The values of Train Loss and Train MAE provide insights into how well your model is performing on the training dataset. Lower values indicate better performance, as they suggest that the model's predictions are closer to the actual ages.
##### It's essential to interpret these values in the context of your specific application and domain knowledge. For instance, consider the scale of the target variable (age) and the implications of prediction errors for your use case.

#### 3.3 Training a Mood Detection Model

In [59]:
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Function to load FER-2013 data
def load_mood_data(data_dir):
    X = []
    y = []

    label_encoder = LabelEncoder()
    labels = []

    # Collect all labels to fit the label encoder
    for emotion in os.listdir(data_dir):
        emotion_dir = os.path.join(data_dir, emotion)
        if os.path.isdir(emotion_dir):
            labels.append(emotion)
    label_encoder.fit(labels)

    print("Labels:", labels)
    print("Label Encoder classes:", label_encoder.classes_)

    # Iterate through each emotion directory
    for emotion in os.listdir(data_dir):
        emotion_dir = os.path.join(data_dir, emotion)
        if os.path.isdir(emotion_dir):
            print(f"Processing directory: {emotion_dir}")
            # Load images from each emotion subdirectory
            for filename in os.listdir(emotion_dir):
                image_path = os.path.join(emotion_dir, filename)
                if os.path.isfile(image_path):
                    # Read image and resize to (48, 48)
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if image is not None:
                        image = cv2.resize(image, (48, 48))
                        # Normalize pixel values to range [0, 1]
                        image = image / 255.0
                        # Encode emotion label
                        label = label_encoder.transform([emotion])[0]
                        # Append image and label to X and y
                        X.append(image)
                        y.append(label)
                    else:
                        print(f"Failed to load image: {image_path}")
                else:
                    print(f"Not a valid file: {image_path}")

    return np.array(X), np.array(y)

# Load the FER-2013 dataset
fer2013_data_dir = r'C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All' # Replace with the path to your FER-2013 dataset directory
X, y = load_mood_data(fer2013_data_dir)

# Check if data is loaded correctly
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Ensure that we have loaded some images
if X.shape[0] == 0 or y.shape[0] == 0:
    raise ValueError("No images loaded. Please check the dataset directory and image paths.")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Expand dimensions to match the input shape of the model
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Define a CNN model for mood detection with dropout layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with validation split and monitor training process
history = model.fit(X_train, y_train, epochs=75, validation_split=0.5)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Save the model
model.save('mood_detection_model.keras')

# Plot training and validation accuracy/loss
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()


Labels: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Label Encoder classes: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\angry
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\disgust
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\fear
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\happy
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\neutral
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\sad
Processing directory: C:\Users\13ics\Bathxpertx\face recognisition\Mood dataset\All\surprise


MemoryError: Unable to allocate 631. MiB for an array with shape (35887, 48, 48) and data type float64

### Step 4: Set Up the Flask Backend
#### 4.1 Create Flask App

In [57]:
# !pip install Flask flask_cors opencv-python tensorflow numpy
# !pip install --upgrade flask flask-cors

In [58]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
import cv2
import numpy as np

app = Flask(__name__)
CORS(app)

# Load models
face_model = tf.keras.models.load_model('face_detection_model.keras')
age_model = tf.keras.models.load_model('age_prediction_model.keras')
mood_model = tf.keras.models.load_model('mood_detection_model.keras')

# Load face detection cascade classifier
face_cascade = cv2.CascadeClassifier('path/to/haarcascade_frontalface_default.xml')

@app.route('/detect', methods=['POST'])
def detect():
    file = request.files['image']
    image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    results = []
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        age = predict_age(face)
        mood = predict_mood(face)
        name = predict_name(face)  # Implement similar to age and mood prediction
        results.append({'age': age, 'mood': mood, 'name': name})

    return jsonify(results)

def predict_age(face):
    face_resized = cv2.resize(face, (64, 64)) / 255.0
    face_resized = np.expand_dims(face_resized, axis=0)
    age = age_model.predict(face_resized)
    return age[0][0]

def predict_mood(face):
    face_resized = cv2.resize(face, (48, 48)) / 255.0
    face_resized = np.expand_dims(face_resized, axis=0)
    mood = mood_model.predict(face_resized)
    return np.argmax(mood)

def predict_name(face):
    face_resized = cv2.resize(face, (224, 224)) / 255.0
    face_resized = np.expand_dims(face_resized, axis=0)
    name = name_model.predict(face_resized)
    return np.argmax(name)

if __name__ == '__main__':
    app.run(debug=True)
    app.run(use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

### Step 5: Create the Frontend
#### 5.1 Basic HTML Structure

### Step 6: Running the Application
#### 6.1 Start the Flask Server

In [52]:
python app.py

SyntaxError: invalid syntax (945115591.py, line 1)

### 6.2 Open the HTML File in Your Browser
#### You can open the HTML file in a web browser to interact with your face detection app.

### Conclusion
#### This guide provides a comprehensive framework for building a face detection application with an attractive UI, including training models and 
#### deploying them with Flask. You can extend this by adding more functionalities and improving the models' accuracy by using larger datasets and 
#### fine-tuning the models.